In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from scipy.spatial.distance import cdist
# from tf.keras.models import Sequential  # This does not work!
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam # - Works

from keras.layers import Dense, GRU, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
data=pd.read_csv("/home/cl-4/2_manthan.csv")
data

,Label,tweet
0,F,@Juliet777777: REVEALED ISIS now occupies ALL...
1,F,@Abunass3r: #ISIS claims to defend Islam yet ...
2,F,@WeTeachLifeSir_: ISIS just bombed a mosque i...
3,F,@xeni: Dude ISIS is bombing Muslim people in ...
4,F,@JohnKStahlUSA: Breaking News: if HRC is elec...
...,...,...
11365,T,@DanieleRaineri: Al Baghdadi sent Husayn Juay...
11366,T,#ISIS Abu Muhammad al-Jninah made his suicide ...
11367,T,"Zahran Alloush once said it:""If we not stormin..."
11368,T,https://www.facebook.com/unsuppoedbrowser


In [3]:
Tweet = np.array(data['tweet'])[:11371]
Label = np.array(data['Label'])[:11371]

In [4]:
data['tweet'].loc[111]

' @jenanmoussa: ISIS attacking the hea of Islam. Targeting Muslims. Next to the prophets mosque. During Ramadan. Just before Eid. #Medin…'

In [5]:
data['Label'].loc[111]

'F'

In [6]:
from collections import Counter

Counter(Label)

Counter({'F': 5685, 'T': 5685})

In [7]:
punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~'
# get rid of punctuation
all_tweet = 'separator'.join(Tweet)
all_tweet = all_tweet.lower()
all_text = ''.join([c for c in all_tweet if c not in punctuation])

# split by new lines and spaces
tweet_split = all_text.split('separator')
all_text = ' '.join(tweet_split)

# create a list of words
words = all_text.split()

In [8]:
data.head(5)

,Label,tweet
0,F,@Juliet777777: REVEALED ISIS now occupies ALL...
1,F,@Abunass3r: #ISIS claims to defend Islam yet ...
2,F,@WeTeachLifeSir_: ISIS just bombed a mosque i...
3,F,@xeni: Dude ISIS is bombing Muslim people in ...
4,F,@JohnKStahlUSA: Breaking News: if HRC is elec...


In [9]:
new_tweet= []
for tweet in tweet_split:
    tweet = tweet.split()
    new_text = []
    for word in Tweet:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_tweet.append(new_text)
  

In [10]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}



In [15]:
print(vocab_to_int)

{'the': 1, 'of': 2, 'and': 3, 'in': 4, 'is': 5, 'to': 6, 'isis': 7, 'a': 8, 'that': 9, 'you': 10, 'muslim': 11, 'they': 12, 'he': 13, 'for': 14, 'allah': 15, 'not': 16, 'said': 17, 'it': 18, 'are': 19, 'will': 20, 'them': 21, 'with': 22, 'from': 23, 'who': 24, 'his': 25, 'islam': 26, 'people': 27, 'no': 28, 'their': 29, 'by': 30, 'this': 31, 'muslims': 32, 'as': 33, 'be': 34, 'him': 35, 'have': 36, 'during': 37, 'on': 38, 'then': 39, 'so': 40, 'has': 41, 'i': 42, 'prophet': 43, 'if': 44, 's': 45, 'month': 46, 'was': 47, 'one': 48, 'we': 49, 'city': 50, 'what': 51, '�': 52, 'those': 53, 'ramadan': 54, 'theyre': 55, 'attack': 56, 'but': 57, 'how': 58, 'religion': 59, 'ibn': 60, 'do': 61, 'holiest': 62, 'holy': 63, 'or': 64, 'bombing': 65, 'your': 66, 'islamic': 67, 'say': 68, 'messenger': 69, 'when': 70, 'were': 71, 'which': 72, 'killed': 73, 'state': 74, 'https…': 75, 'just': 76, 'all': 77, 'mosque': 78, 'than': 79, 'would': 80, 'ever': 81, 'us': 82, 'against': 83, 'an': 84, '@xeni': 85

In [12]:
tweet_ints = []


In [14]:
for tweet in new_tweet:
    tweet_ints.append([vocab_to_int[word] for word in tweet])


KeyError: " @WeTeachLifeSir_: ISIS just bombed a mosque in the 2nd holiest city in Islam during the holiest month in Islam.   Don't ever say that IS…"

In [ ]:
print('Unique words: ', len((vocab_to_int)))  # should ~ 74000+
print()


In [ ]:
print('Tokenized tweet: \n', tweet_ints[:1])


In [ ]:
# T - Terrorits & F - Not a terrorits
encoded_labels = []

encoded_labels = []
for label in Label:
    if label == 'F':
        encoded_labels.append(0)
    else:
        encoded_labels.append(1)

encoded_labels = np.asarray(encoded_labels)

In [ ]:
#Padding Sequence

In [ ]:
def pad_features(tweet_ints, seq_length):
    
    # getting the correct rows x cols shape
    features = np.zeros((len(tweet_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(tweet_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

In [ ]:
seq_length = 30

features = pad_features(tweet_ints, seq_length=seq_length)

## test statements 
assert len(features)==len(tweet_ints), "The features should have as many rows as tweet."
assert len(features[0])==seq_length, "Each feature row should contain seq_length values."

# print first 10 values of the first 30 batches 
print(features[:19,:19])

In [ ]:
# ""Training, validation, and test"""''"""


In [ ]:
split_frac = 0.8

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of the resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

# dataloaders
batch_size = 50

# make sure the SHUFFLE the training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [ ]:
pip install torch


In [ ]:
# obtain one batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

In [ ]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

In [ ]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 200
hidden_dim = 128
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

In [ ]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
# training params

#                              CHANGE AFTERWARDS
epochs = 2 

counter = 0
print_every = 100
clip=5 # gradient clipping

# move model to GPU, if available
if(train_on_gpu):
    net.cuda()

net.train()
# train for some number of epochs
for e in range(epochs):
    # initialize hidden state
    h = net.init_hidden(batch_size)

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()

        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()

                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())

                val_losses.append(val_loss.item())

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [ ]:
# Get test data loss and accuracy

test_losses = [] # track loss
num_correct = 0

# init hidden state
h = net.init_hidden(batch_size)

net.eval()
# iterate over test data
for inputs, labels in test_loader:

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in h])

    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # get predicted outputs
    output, h = net(inputs, h)
    
    # calculate loss
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze())  # rounds to the nearest integer
    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)


# -- stats! -- ##
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

In [ ]:
# Creating a prediction function & saving model is remianing 